# Dual-Retriever & Relational Expansion Demo

This notebook demonstrates **Step 1: The Dual-Retriever**. 

Unlike standard RAG, VeNRA connects structured data (UFL) and unstructured text (Chunks) bi-directionally using **Relational Expansion** (GraphRAG-style).

### Verification Workflow:
1. **Full Ingestion:** Run the `IngestionPipeline` to ensure `ufl.parquet` and the vector index are persisted.
2. **Clue Generation:** Use the Navigator to generate `RetrievalPlan` (Clues).
3. **Dual-Retrieval:** Execute the search and observe the relational merging of rows and text.

In [1]:
%load_ext autoreload
%autoreload 2

import sys
import os
import json
import nest_asyncio
from dotenv import load_dotenv

# Add src to path
sys.path.append(os.path.abspath("../src"))
load_dotenv("../.env")

from venra.pipeline import IngestionPipeline
from venra.navigator import Navigator
from venra.retriever import DualRetriever
from venra.logging_config import logger

nest_asyncio.apply()

/Users/pedram/Projects/VeNRA/.venv/lib/python3.11/site-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'validate_default' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'validate_default' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(


## 1. Run Pipeline (Ensures Data Persistence)

This will parse the TransDigm 10-K, melt tables into the UFL, and index everything.

In [2]:
PDF_PATH = "../data/10K_TD_test.pdf"
pipeline = IngestionPipeline()

# Run the full pipeline to create ufl.parquet and index chunks
await pipeline.run(PDF_PATH, skip_parsing=True)

2026-02-03 14:48:49,005 - venra - INFO - UFL already exists at /Users/pedram/Projects/VeNRA/data/processed/10K_TD_test_ufl.parquet. Skipping extraction.


[UFLRow(row_id='e65db3dc53e8ddc0d839204d603903a7', entity_id='ID_TDG', entity_name_raw='TransDigm Group Incorporated', metric_name='Revenue', value=500000000.0, unit='USD', scale_factor=1.0, period='2025', doc_section='☒ Annual Report Pursuant to Section 13 or 15(d) of the Securities Exchange Act of 1934', source_chunk_id='f1bb65a4-9938-465d-8a10-db1c7f16795f', nuance_note=None, confidence=1.0, related_entity_id=None),
 UFLRow(row_id='5cf22b20665502f78ad48a3aee1d2a11', entity_id='ID_TDG', entity_name_raw='TransDigm Group Incorporated', metric_name='Interest Rate', value=5.0, unit='Percent', scale_factor=1.0, period='2025', doc_section='☒ Annual Report Pursuant to Section 13 or 15(d) of the Securities Exchange Act of 1934', source_chunk_id='f1bb65a4-9938-465d-8a10-db1c7f16795f', nuance_note=None, confidence=1.0, related_entity_id=None),
 UFLRow(row_id='0ed88d45198ecaf781aedf3a1af9018c', entity_id='ID_TDG', entity_name_raw='TransDigm Group Incorporated', metric_name='Acquisitions', value

## 2. Initialize Components

In [3]:
file_prefix = os.path.basename(PDF_PATH).replace(".pdf", "")
nav = Navigator(file_prefix=file_prefix)
retriever = DualRetriever(file_prefix=file_prefix)

2026-02-03 14:49:34,559 - venra - INFO - Retriever loaded UFL with 253 rows.


## 3. Clue Generation -> Hybrid Retrieval

We ask about acquisitions. The Navigator provides the clues, and the Retriever executes both branches.

In [6]:
query = "How much the sale excluding acquisition increase compare to last year and what it was due to?"
plan = await nav.navigate(query)

print(f"--- Clues for: {query} ---")
print(f"Reasoning: {plan.reasoning}")
if plan.ufl_query:
    print(f"UFL Keywords: {plan.ufl_query.metric_keywords}")
print(f"Hypothesis:   {plan.vector_hypothesis}")

# Execute Retrieval with expansion enabled
results = await retriever.retrieve(
    plan, 
    k=3, 
    include_all_chunks_for_ufl=True, 
    include_all_ufl_for_chunks=True
)

print(f"\n--- Results ---")
print(f"Total UFL Rows: {len(results['ufl_rows'])}")
print(f"Total Chunks:   {len(results['text_chunks'])}")

print(f"\n--- Sample UFL Rows ---")
for r in results["ufl_rows"][:5]:
    print(f"- {r.metric_name} ({r.period}): {r.value} {r.unit}")

2026-02-03 14:56:39,743 - venra - INFO - Navigating query: How much the sale excluding acquisition increase compare to last year and what it was due to?
2026-02-03 14:56:40,423 - venra - INFO - Plan generated. Reasoning: The user is asking for the increase in net sales excluding acquisition compared to last year and what it was due to. This requires looking at the entity ID 'ID_TDG' and metrics 'Net Sales', 'Acquisition Sales', and 'Increase in Organic Sales' for the years '2024' and '2023'.
--- Clues for: How much the sale excluding acquisition increase compare to last year and what it was due to? ---
Reasoning: The user is asking for the increase in net sales excluding acquisition compared to last year and what it was due to. This requires looking at the entity ID 'ID_TDG' and metrics 'Net Sales', 'Acquisition Sales', and 'Increase in Organic Sales' for the years '2024' and '2023'.
UFL Keywords: ['Net Sales', 'Acquisition Sales', 'Increase in Organic Sales']
Hypothesis:   Net Sales E

In [9]:
results['ufl_rows']

[UFLRow(row_id='c7ff6dac0728f1b924f80b5f1f504c56', entity_id='ID_TDG', entity_name_raw='TransDigm Group Incorporated', metric_name='Net Sales', value=500000000.0, unit='USD', scale_factor=1.0, period='2024', doc_section='Total Company', source_chunk_id='9d7492d3-c7c0-44b7-b666-f112cb7ff135', nuance_note=None, confidence=1.0, related_entity_id=None),
 UFLRow(row_id='8c51683e431208c414e06943746ab253', entity_id='ID_TDG', entity_name_raw='TransDigm Group Incorporated', metric_name='Acquisition Sales', value=nan, unit=None, scale_factor=1.0, period='2024', doc_section='Total Company', source_chunk_id='9d7492d3-c7c0-44b7-b666-f112cb7ff135', nuance_note=None, confidence=1.0, related_entity_id=None),
 UFLRow(row_id='10744edf2fc939570d3c6d833158df41', entity_id='ID_TDG', entity_name_raw='TransDigm Group Incorporated', metric_name='Net Sales', value=500000000.0, unit='USD', scale_factor=1.0, period='2024', doc_section='Business Segments', source_chunk_id='0adc8287-2fc7-411d-ab57-9e5e65da4965', 